In [1]:
!pip install jsonlines




In [2]:
import jsonlines
import sys
import re
import nltk
nltk.download('averaged_perceptron_tagger')
### import nltk POS tagger here
import nltk

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alphaeus Talks\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
inputFileLocation = 'english-jsonl/test.json'
outputFileLocation = 'english-bio/test.bio'
tagTableFileLocation = 'tagtable-English-2022.txt'
inputFileLocation = 'english-jsonl/train.json'
outputFileLocation = 'english-bio/train.bio'
tagTableFileLocation = 'tagtable-English-2022.txt'

In [17]:
# get annotation name to BIO tag table
tagTable = {}
tagTableTemp = open(tagTableFileLocation).read().split('\n')
for tag in tagTableTemp:
    if len(tag):
        tag = tag.split('|')
        tagTable[tag[0]] = tag[1]

In [18]:
# go through document and make BIO output
annotationId = 0

bioOutput = ''
with jsonlines.open(inputFileLocation) as reader:
    for obj in reader:
        if len(obj['label']):
            # first, make list of annotated characters
            annotatedCharacters = {} 
            for ann in obj['label']: 
                annotationId += 1
                annRange = range(ann[0], ann[1]+1) # make range, plus 1 at the end as we want to include the space after the word
                #print(annRange)
                #print(obj['text'][ann[0]:ann[1]+1])
                for number in annRange:
                    annotatedCharacters[number] = {'label' : tagTable[ann[2]], 'id' : annotationId} # add all numbers in range to annchars, so we can check each space against it later

            # now we'll check each space in string for a match against annotatedCharacters
            bioTags = []
            i = 0
            previousAnnId = 0
            text = obj['text']+'  ' # add space at end so we can annotate last word
            for char in text:
                #print(char+obj['text'][i])
                if i > 0:
                    #print(i)
                    if char == " " and text[i-1] != " ": # second part to stop double spaces
                        if i in annotatedCharacters and (i-1) in annotatedCharacters: # this space is at the end of an annotated word
                            tag = annotatedCharacters[i]['label']
                            if previousAnnId is annotatedCharacters[i]['id']:
                                BorI = "I-"
                            else:
                                BorI = "B-"
                            bioTags.append(BorI+tag)
                            previousAnnId = annotatedCharacters[i]['id']
                        else:
                            bioTags.append('O')
                i += 1
        else:
            bioTags = ['O'] * len(obj['text'].split('  '))

        # add POS tags via NLTK      
        output = nltk.pos_tag(obj['text'].split())
        i = 0
        outputParagraph = ''
        for token in output:
            outputParagraph += token[0]+' '+token[1]+' '+bioTags[i]+'\n'
            i += 1

        bioOutput += outputParagraph+'\n'
    
        #break

with open(outputFileLocation, "wb") as text_file:
    text_file.write(bioOutput.encode('utf-8', errors='ignore'))
